<a href="https://colab.research.google.com/github/ShreyPandit/Lipschitz/blob/master/CIFAR10_RMSProp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np
from tqdm import tqdm
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, BatchNormalization, MaxPool2D, Dropout
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

from tqdm import tqdm_notebook as tqdm

import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')
!pip install tensorflow==1.14.0

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [3]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [4]:
batch_size = 256
num_classes = 10
epochs = 20
img_rows, img_cols = 32, 32
lrs=[]

In [5]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

In [15]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal',input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.20))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)         

In [7]:
from keras import backend as K
func = K.function([model.layers[0].input], [model.layers[-2].output])

In [8]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [16]:
lrs = []
K2 = 0.
beta2 = 0.9

In [17]:
def lr_schedule(epoch):
    global K2

    Kz = 0.  # max penultimate activation
    S = 0.
    
    sess = K.get_session()
    max_wt = 0.
    for weight in model.weights:
        norm = np.linalg.norm(weight.eval(sess))
        if norm > max_wt:
            max_wt = norm
    
    for i in tqdm(range((len(x_train) - 1) // batch_size + 1)):
        start_i = i * batch_size
        end_i = start_i + batch_size
        xb = x_train[start_i:end_i]
    	
        tmp = np.array(func([xb]))
        activ = np.linalg.norm(tmp)
        sq = np.linalg.norm(np.square(tmp))

        if sq > S:
            S = sq
        
        if activ > Kz:
            Kz = activ

    S_ = (num_classes - 1) ** 2 / (num_classes * batch_size) ** 2 * S + 2e-4 * max_wt * ((num_classes - 1) * Kz) / (num_classes * batch_size)
    
    K2 = beta2 * K2 + (1 - beta2) * S_

    lr = (np.sqrt(K2) + K.epsilon()) / Kz
    print('S =', S, 'max_wt =', max_wt, 'Kz =', Kz, 'K2 =', K2, 'S_ =', S_)
    lrs.append(lr)
    print('Epoch', epoch+1, 'LR =', lr)
    return lr


In [18]:
lr_scheduler = LearningRateScheduler(lr_schedule)

In [19]:
from keras.optimizers import RMSprop
optimizer = RMSprop(lr=1e-3, decay=1e-4)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=["accuracy"])

In [20]:
datagen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False, 
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=15,
        zoom_range = 0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)

datagen.fit(x_train)

In [21]:
history=model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train) / batch_size,
                    epochs=15,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[lr_scheduler])

Epoch 1/15


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



S = 330.66135 max_wt = 15.985094 Kz = 157.15642 K2 = 0.0005853210892990343 S_ = 0.005853210892990344
Epoch 1 LR = 0.000153945415466705
196/195 [==============================] - 290s 1s/step - loss: 2.1206 - accuracy: 0.2679 - val_loss: 1.6360 - val_accuracy: 0.4059
Epoch 2/15



S = 330.66135 max_wt = 15.993725 Kz = 157.15642 K2 = 0.0011122054400072726 S_ = 0.00585416459638142
Epoch 2 LR = 0.00021220796887563368
196/195 [==============================] - 283s 1s/step - loss: 1.6462 - accuracy: 0.4005 - val_loss: 1.4307 - val_accuracy: 0.4700
Epoch 3/15



S = 330.66135 max_wt = 16.06892 Kz = 157.15642 K2 = 0.0015872322684068923 S_ = 0.005862473724003471
Epoch 3 LR = 0.000253506565214698
196/195 [==============================] - 279s 1s/step - loss: 1.4863 - accuracy: 0.4637 - val_loss: 1.4915 - val_accuracy: 0.4735
Epoch 4/15


...


S = 330.66135 max_wt = 18.22722 Kz = 157.15642 K2 = 0.00410138206500648 S_ = 0.006100967226753371
Epoch 11 LR = 0.00040750568419068085
196/195 [==============================] - 302s 2s/step - loss: 1.0048 - accuracy: 0.6451 - val_loss: 0.8665 - val_accuracy: 0.7050
Epoch 12/15



S = 330.66135 max_wt = 18.56248 Kz = 157.15642 K2 = 0.0043050452288311055 S_ = 0.0061380137032527315
Epoch 12 LR = 0.00041750088461979334
196/195 [==============================] - 295s 2s/step - loss: 0.9695 - accuracy: 0.6581 - val_loss: 0.8809 - val_accuracy: 0.7017
Epoch 13/15



S = 330.66135 max_wt = 18.902088 Kz = 157.15642 K2 = 0.004492094756852006 S_ = 0.006175540509040106
Epoch 13 LR = 0.000426474414087481
196/195 [==============================] - 278s 1s/step - loss: 0.9460 - accuracy: 0.6687 - val_loss: 1.0974 - val_accuracy: 0.6421
Epoch 14/15



S = 330.66135 max_wt = 19.24497 Kz = 157.15642 K2 = 0.00466422820068762 S_ = 0.006213429195208139
Epoch 14 LR = 0.0004345686671064024
196/195 [==============================] - 280s 1s/step - loss: 0.9214 - accuracy: 0.6766 - val_loss: 0.8046 - val_accuracy: 0.7243
Epoch 15/15



S = 330.66135 max_wt = 19.584244 Kz = 157.15642 K2 = 0.0048228972923627525 S_ = 0.006250919117438953
Epoch 15 LR = 0.00044189848323516833
196/195 [==============================] - 281s 1s/step - loss: 0.8973 - accuracy: 0.6851 - val_loss: 1.1119 - val_accuracy: 0.6371
